# MCP + LangGraph 핸즈온 튜토리얼

- 작성자: [테디노트](https://youtube.com/c/teddynote)
- 강의: [패스트캠퍼스 RAG 비법노트](https://fastcampus.co.kr/data_online_teddy)

**참고자료**
- https://modelcontextprotocol.io/introduction
- https://github.com/langchain-ai/langchain-mcp-adapters

## 환경설정

아래 설치 방법을 참고하여 `uv` 를 설치합니다.

**uv 설치 방법**

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# Windows (PowerShell)
irm https://astral.sh/uv/install.ps1 | iex
```

**의존성 설치**

```bash
uv pip install -r requirements.txt
```

환경변수를 가져옵니다.

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)
logging.langsmith("LangGraph MCP Adapters HandsOn")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangGraph MCP Adapters HandsOn


## MultiServerMCPClient

사전에 `mcp_server_remote.py` 를 실행해둡니다. 터미널을 열고 가상환경이 활성화 되어 있는 상태에서 서버를 실행해 주세요.

> 명령어
```bash
source .venv/bin/activate
python mcp_server_remote.py
```

`async with` 로 일시적인 Session 연결을 생성 후 해제

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import ainvoke_graph, astream_graph
# from langchain_anthropic import ChatAnthropic

# model = ChatAnthropic(
#     model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
# )
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    api_key="ollama",
    model="llama3.2:1b",
    base_url="http://localhost:11434/v1",
    temperature=0,
)

async with MultiServerMCPClient(
    {
        "weather": {
            # 서버의 포트와 일치해야 합니다.(8005번 포트)
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x136457e20>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in Seoul
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - 

다음의 경우에는 session 이 닫혔기 때문에 도구에 접근할 수 없는 것을 확인할 수 있습니다.

In [3]:
await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Error: ClosedResourceError()
 Please fix your mistakes.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
<|python_tag|>import requests

def get_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid=YOUR_OPENWEATHERMAP_API_KEY"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        weather_description = data["weather"][0]["description"]
        temperature = f"{data['main']['temp']}°C"
        
        return f"서울의 날씨는 {weather_description}입니다. 기온은 {temperature}°C입니다."
    else:
        return "Failed to retrieve weather information."

print(get_weather("Seoul"))

{'node': 'agent',
 'content': AIMessageChunk(content='<|python_tag|>import requests\n\ndef get_weather(location):\n    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid=YOUR_OPENWEATHERMAP_API_KEY"\n    response = requests.get(url)\n    \n    if response.status_code == 200:\n        data = response.json()\n        weather_description = data["weather"][0]["description"]\n        temperature = f"{data[\'main\'][\'temp\']}°C"\n        \n        return f"서울의 날씨는 {weather_description}입니다. 기온은 {temperature}°C입니다."\n    else:\n        return "Failed to retrieve weather information."\n\nprint(get_weather("Seoul"))', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama3.2:1b', 'system_fingerprint': 'fp_ollama'}, id='run-3dc1f344-3a4b-466b-8780-61a299aeaadd'),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),


이제 그럼 Async Session 을 유지하며 도구에 접근하는 방식으로 변경해 보겠습니다.

In [4]:
# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)


# 2. 명시적으로 연결 초기화 (이 부분이 필요함)
# 초기화
await client.__aenter__()

# 이제 도구가 로드됨
print(client.get_tools())  # 도구가 표시됨

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x137dc51c0>)]


Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


langgraph 의 에이전트를 생성합니다.

In [5]:
# 에이전트 생성
agent = create_react_agent(model, client.get_tools())

그래프를 실행하여 결과를 확인합니다.

In [6]:
await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in Seoul
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
I'm not sure that's the best response to a weather question. While it's true that Seoul is known for its sunny weather, I should provide more accurate and relevant information.

Let me try again. How about this:

"Seoul's weather is usually mild and pleasant, with average high temperatures in the mid-20s to low 30s Celsius (mid-70s to mid-90s Fahrenheit) during the summer months."

Please let me know if you'd like more specific or up-to-date information on Seoul's weather.

{'node': 'agent',
 'content': AIMessageChunk(content='I\'m not sure that\'s the best response to a weather question. While it\'s true that Seoul is known for its sunny weather, I should provide more accurate and relevant information.\n\nLet me try again. How about this:\n\n"Seoul\'s weather is usually mild and pleasant, with average high temperatures in the mid-20s to low 30s Celsius (mid-70s to mid-90s Fahrenheit) during the summer months."\n\nPlease let me know if you\'d like more specific or up-to-date information on Seoul\'s weather.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama3.2:1b', 'system_fingerprint': 'fp_ollama'}, id='run-7b359c80-7d7b-4bda-a57c-6ddfc0dc906c'),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:b1a20969-d004-8f87-0581-f0517ad59cab',
  'checkpoint_ns': 

## Stdio 통신 방식

Stdio 통신 방식은 로컬 환경에서 사용하기 위해 사용합니다.

- 통신을 위해 표준 입력/출력 사용

참고: 아래의 python 경로는 수정하세요!

In [8]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_anthropic import ChatAnthropic

# LLM 모델 초기화
model = ChatOpenAI(
    api_key="ollama",
    model="llama3.2:1b",
    base_url="http://localhost:11434/v1",
    temperature=0,
)
# StdIO 서버 파라미터 설정
# - command: Python 인터프리터 경로
# - args: 실행할 MCP 서버 스크립트
server_params = StdioServerParameters(
    command="/Users/hyunoozzing/.pyenv/versions/3.11.11/envs/llm/bin/python",
    args=["mcp_server_local.py"],
)

# StdIO 클라이언트를 사용하여 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드
        tools = await load_mcp_tools(session)
        print(tools)

        # 에이전트 생성
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1405780e0>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in Seoul
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - 

## RAG 를 구축한 MCP 서버 사용

- 파일: `mcp_server_rag.py`

사전에 langchain 으로 구축한 `mcp_server_rag.py` 파일을 사용합니다.

stdio 통신 방식으로 도구에 대한 정보를 가져옵니다. 여기서 도구는 `retriever` 도구를 가져오게 되며, 이 도구는 `mcp_server_rag.py` 에서 정의된 도구입니다. 이 파일은 사전에 서버에서 실행되지 **않아도** 됩니다.

In [10]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
from langchain_teddynote.messages import astream_graph

# LLM 모델 초기화
model = ChatOpenAI(
    api_key="ollama",
    model="llama3.2:1b",
    base_url="http://localhost:11434/v1",
    temperature=0,
)

# RAG 서버를 위한 StdIO 서버 파라미터 설정
server_params = StdioServerParameters(
    command="/Users/hyunoozzing/.pyenv/versions/3.11.11/envs/llm/bin/python",
    args=["./mcp_server_rag.py"],
)

# StdIO 클라이언트를 사용하여 RAG 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드 (여기서는 retriever 도구)
        tools = await load_mcp_tools(session)

        # 에이전트 생성 및 실행
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(
            agent, {"messages": "삼성전자가 개발한 생성형 AI의 이름을 검색해줘"}
        )


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
2
G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의
n G7이 첨단 AI 시스템을 개발하는 기업을 대상으로 AI 위험 식별과 완화를 위해 자발적인 
채택을 권고하는 AI 국제 행동강령을 마련
n 행동강령은 AI 수명주기 전반에 걸친 위험 평가와 완화, 투명성과 책임성의 보장, 정보공유와 
이해관계자 간 협력, 보안 통제, 콘텐츠 인증과 출처 확인 등의 조치를 요구
KEY Contents
£ G7, 첨단 AI 시스템의 위험 관리를 위한 국제 행동강령 마련
n 주요 7개국(G7)*은 2023년 10월 30일 ‘히로시마 AI 프로세스’를 통해 AI 기업 대상의 AI 국제 
행동강령(International Code of Conduct for Advanced AI Systems)에 합의
∙G7은 2023년 5월 일본 히로시마에서 개최된 정상회의에서 생성 AI에 관한 국제규범 마련과 
정보공유를 위해 ‘히로시마 AI 프로세스’를 출범**
∙기업의 자발적 채택을 위해 마련된 이번 행동강령은 기반모델과 생성 AI를 포함한 첨단 AI 시스템의 
위험 식별과 완화에 필요한 조치를 포함
* 주요 7개국(G7)은 미국, 일본, 독일, 영국, 프랑스, 이탈리아, 캐나다를 의미
** 5월 정상회의에는 한국, 호주, 베트남 등을 포함한 8개국이 초청을 받았으나, AI 국제 행동강령에는 우선 G7 국가만 포함하여 채택
n G7은 행동강령을 통해 아래의 조치를 제시했으며, 빠르게 발전하는 기술에 대응할 수 있도록 
이해관계자 협의를 통해 필요에 따라 개정할 예정
∙첨단 AI 시스템의 개발 과정에서 AI 수명주기 전반에 걸쳐 위험을 평가 및 완화하는 조치를 채택하고, 
첨단 

## SSE 방식과 StdIO 방식 혼합 사용

- 파일: `mcp_server_rag.py` 는 StdIO 방식으로 통신
- `langchain-dev-docs` 는 SSE 방식으로 통신

SSE 방식과 StdIO 방식을 혼합하여 사용합니다.

In [11]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# LLM 모델 초기화
model = ChatOpenAI(
    api_key="ollama",
    model="llama3.2:1b",
    base_url="http://localhost:11434/v1",
    temperature=0,
)

# 1. 다중 서버 MCP 클라이언트 생성
client = MultiServerMCPClient(
    {
        "document-retriever": {
            "command": "/Users/hyunoozzing/.pyenv/versions/3.11.11/envs/llm/bin/python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
        "langchain-dev-docs": {
            # SSE 서버가 8765 포트에서 실행 중인지 확인하세요
            "url": "http://teddynote.io:8765/sse",
            # SSE(Server-Sent Events) 방식으로 통신
            "transport": "sse",
        },
    }
)


# 2. 비동기 컨텍스트 매니저를 통한 명시적 연결 초기화
await client.__aenter__()

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [12]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = (
    "You are a smart agent. "
    "Use `retriever` tool to search on AI related documents and answer questions."
    "Use `langchain-dev-docs` tool to search on langchain / langgraph related documents and answer questions."
    "Answer in Korean."
)
agent = create_react_agent(
    model, client.get_tools(), prompt=prompt, checkpointer=MemorySaver()
)

구축해 놓은 `mcp_server_rag.py` 에서 정의한 `retriever` 도구를 사용하여 검색을 수행합니다.

In [13]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
["Below is the list of references for the answer to the question.(Ordered by relevance descending):", "1. https://wikidocs.net/262612", "2. https://wikidocs.net/234281", "3. https://wikidocs.net/262595", "4. https://wikidocs.net/251190"]
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자의 생성형 AI 이름은 다음과 같은 tên이 있습니다.

1. LLaMA
2. BERT
3. T5
4. XLNet

{'node': 'agent',
 'content': AIMessageChunk(content='삼성전자의 생성형 AI 이름은 다음과 같은 tên이 있습니다.\n\n1. LLaMA\n2. BERT\n3. T5\n4. XLNet', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama3.2:1b', 'system_fingerprint': 'fp_ollama'}, id='run-f745e031-b2cf-484e-a3c0-55b161cb365c'),
 'metadata': {'thread_id': 1,
  'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:707a65c3-dd3c-adad-3468-21be4286f3fa',
  'checkpoint_ns': 'agent:707a65c3-dd3c-adad-3468-21be4286f3fa',
  'ls_provider': 'openai',
  'ls_model_name': 'llama3.2:1b',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0}}

이번에는 `langchain-dev-docs` 도구를 사용하여 검색을 수행합니다.

In [14]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {"messages": "langgraph-dev-docs 참고해서 self-rag 의 정의에 대해서 알려줘"},
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
["Below is the list of references for the answer to the question.(Ordered by relevance descending):", "1. https://wikidocs.net/234009", "2. https://wikidocs.net/234017", "3. https://wikidocs.net/233350", "4. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/"]
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
self-rag는 Language Graph AI (LangGraph)에서 Self-RAG (Self-Generated Abstract Representation)이라는 concept을 정의한 AI 기술입니다.

Self-RAG은 Language Model이 그 자체의 문장이나 문맥을 학습하고, 그 자체를 통해 새로운 문장을 생성할 수 있는 기법입니다. self-rag는 Language Graph AI에서 사용되는 Self-Generated Abstract Representation (SRA) technology의 한 형태로, Language Model이 그 자체의 문장이나 문맥을 학습하고, 그 자체를 통해 새로운 문장을 생성할 수 있도록 도와줍니다.

self-rag은 Language Model이 그 자체를 통해 새로운 문장을 생성할 수 있는 기법으로, Self-Generated Abstract Representation (SRA) technology의 한 형태입니다.

{'node': 'agent',
 'content': AIMessageChunk(content='self-rag는 Language Graph AI (LangGraph)에서 Self-RAG (Self-Generated Abstract Representation)이라는 concept을 정의한 AI 기술입니다.\n\nSelf-RAG은 Language Model이 그 자체의 문장이나 문맥을 학습하고, 그 자체를 통해 새로운 문장을 생성할 수 있는 기법입니다. self-rag는 Language Graph AI에서 사용되는 Self-Generated Abstract Representation (SRA) technology의 한 형태로, Language Model이 그 자체의 문장이나 문맥을 학습하고, 그 자체를 통해 새로운 문장을 생성할 수 있도록 도와줍니다.\n\nself-rag은 Language Model이 그 자체를 통해 새로운 문장을 생성할 수 있는 기법으로, Self-Generated Abstract Representation (SRA) technology의 한 형태입니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama3.2:1b', 'system_fingerprint': 'fp_ollama'}, id='run-36e64991-01c9-4a99-a504-f81bc5df97d3'),
 'metadata': {'thread_id': 1,
  'langgraph_step': 8,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:0812fb44-3cd5-e89b-2836-e8ed39da

`MemorySaver` 를 사용하여 단기 기억을 유지합니다. 따라서, multi-turn 대화도 가능합니다.

In [15]:
await astream_graph(
    agent, {"messages": "이전의 내용을 bullet point로 요약해줘"}, config=config
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
["Below is the list of references for the answer to the question.(Ordered by relevance descending):", "1. https://wikidocs.net/262612", "2. https://wikidocs.net/234281", "3. https://wikidocs.net/262595", "4. https://wikidocs.net/251190"]["Below is the list of references for the answer to the question.(Ordered by relevance descending):", "1. https://wikidocs.net/234009", "2. https://wikidocs.net/234017", "3. https://wikidocs.net/233350", "4. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/"]["Below is the list of references for the answer to the question.(Ordered by relevance descending):", "1. https://wikidocs.net/234009", "2. https://wikidocs.net/234017", "3. https://wikidocs.net/233350", "4. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/"]
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - 

{'node': 'agent',
 'content': AIMessageChunk(content='삼성전자의 생성형 AI 이름은 다음과 같이 정의된 self-rag입니다.\n\n*   Self-RAG (Self-Generated Abstract Representation)\n*   Language Graph AI (LangGraph)에서 Self-Generated Abstract Representation (SRA) technology의 한 형태\n*   Language Model이 그 자체를 통해 새로운 문장을 생성할 수 있는 기법\n\nself-rag은 Language Model이 그 자체를 통해 새로운 문장을 생성할 수 있도록 도와줍니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama3.2:1b', 'system_fingerprint': 'fp_ollama'}, id='run-0ee30259-440c-4a32-9392-91b9b51fbdde'),
 'metadata': {'thread_id': 1,
  'langgraph_step': 13,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:175429e7-113f-0aec-58c5-907ca041b8fa',
  'checkpoint_ns': 'agent:175429e7-113f-0aec-58c5-907ca041b8fa',
  'ls_provider': 'openai',
  'ls_model_name': 'llama3.2:1b',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0}}

## LangChain 에 통합된 도구 + MCP 도구

여기서는 LangChain 에 통합된 도구를 기존의 MCP 로만 이루어진 도구와 함께 사용이 가능한지 테스트 합니다.

In [16]:
from langchain_teddynote.tools.tavily import TavilySearch

# Tavily 검색 도구를 초기화 합니다. (news 타입, 최근 3일 내 뉴스)
tavily = TavilySearch(max_results=3, topic="news", days=3)

# 기존의 MCP 도구와 함께 사용합니다.
tools = client.get_tools() + [tavily]

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [17]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = "You are a smart agent with various tools. Answer questions in Korean."
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=MemorySaver())

새롭게 추가한 `tavily` 도구를 사용하여 검색을 수행합니다.

In [18]:
await astream_graph(agent, {"messages": "오늘 뉴스 찾아줘"}, config=config)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[{"url": "https://medicalxpress.com/news/2025-03-highly-accurate-blood-alzheimer-disease.html", "raw_content": "Published Time: 2025-03-31T11:34:05-04:00\nHighly accurate blood test diagnoses Alzheimer's disease, measures extent of dementia\nTopics\nConditions\nWeek's top\nLatest news\nUnread news\nSubscribe\nScience X Account\nRemember me\nSign In\nClick here to sign in with or\nForget Password?\nNot a member? Sign up\nLearn more\nAddiction\nAlzheimer's disease & dementia\nArthritis & Rheumatism\nAttention deficit disorders\nAutism spectrum disorders\nBiomedical technology\nCardiology\nDentistry\nDiabetes\nDiseases, Conditions, Syndromes\nEndocrinology & Metabolism\nGastroenterology\nGenetics\nGerontology & Geriatrics\nHealth\nHealth informatics\nHIV & AIDS\nImmunology\nInflammatory disorders\nMedical economics\nMedical research\nMedications\nNeurosci

{'node': 'agent',
 'content': AIMessageChunk(content='The article discusses the latest developments in the world of politics and technology. Here are some key points:\n\n* A coyote has been found to be living in a Warner Bros. water tower, sparking debate about animal welfare and property rights.\n* The coyote, named "Coyote vs. Acme," was discovered by a group of filmmakers who were filming a scene for an upcoming movie.\n* The incident highlights the challenges of balancing artistic freedom with animal welfare concerns.\n* The film industry has been criticized for its treatment of animals in the past, and this incident is seen as a wake-up call to consider the ethics of using real animals in productions.\n\nThe article also mentions that the coyote\'s owner, who was not named, had previously reported the animal to local authorities. However, it appears that the coyote has been living in the water tower for some time, and its exact circumstances are unclear.\n\nOverall, the article ra

`retriever` 도구가 원활하게 작동하는 것을 확인할 수 있습니다.

In [19]:
await astream_graph(
    agent,
    {
        "messages": "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
["Below is the list of references for the answer to the question.(Ordered by relevance descending):", "1. https://wikidocs.net/262612", "2. https://wikidocs.net/234281", "3. https://wikidocs.net/262595", "4. https://wikidocs.net/251190"]
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI 이름은 다음과 같습니다.

1. **Nexarion**: Nexarion은 삼성전자의 생성형 AI 이름 중 하나입니다. nó는 생성형 AI의 특징을 강조하며, 생성형 AI를 사용하여 다양한 문제를 해결할 수 있습니다.
2. **Genesis**: Genesis은 다른 생성형 AI 이름 중 하나입니다. nó는 생성형 AI를 사용하여 humans와 machines의 상호작용을 개선할 수 있습니다.
3. **Aurora**: Aurora은 다른 생성형 AI 이름 중 하나입니다. nó는 생성형 AI를 사용하여 humans과 machines의 상호작용을 개선할 수 있습니다.
4. **Lumina**: Lumina은 다른 생성형 AI 이름 중 하나입니다. nó는 생성형 AI를 사용하여 humans과 machines의 상호작용을 개선할 수 있습니다.

이러한 생성형 AI 이름은 삼성전자가 개발한 생성형 AI 이름입니다.

{'node': 'agent',
 'content': AIMessageChunk(content='삼성전자가 개발한 생성형 AI 이름은 다음과 같습니다.\n\n1. **Nexarion**: Nexarion은 삼성전자의 생성형 AI 이름 중 하나입니다. nó는 생성형 AI의 특징을 강조하며, 생성형 AI를 사용하여 다양한 문제를 해결할 수 있습니다.\n2. **Genesis**: Genesis은 다른 생성형 AI 이름 중 하나입니다. nó는 생성형 AI를 사용하여 humans와 machines의 상호작용을 개선할 수 있습니다.\n3. **Aurora**: Aurora은 다른 생성형 AI 이름 중 하나입니다. nó는 생성형 AI를 사용하여 humans과 machines의 상호작용을 개선할 수 있습니다.\n4. **Lumina**: Lumina은 다른 생성형 AI 이름 중 하나입니다. nó는 생성형 AI를 사용하여 humans과 machines의 상호작용을 개선할 수 있습니다.\n\n이러한 생성형 AI 이름은 삼성전자가 개발한 생성형 AI 이름입니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama3.2:1b', 'system_fingerprint': 'fp_ollama'}, id='run-c71fc97a-80b0-47ed-a7c8-b9c1d5f5a53c'),
 'metadata': {'thread_id': 1,
  'langgraph_step': 8,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:11eaa16a-c9c5-6aec-d280-60a6329d3ee5',
  'checkp

## Smithery 에서 제공하는 MCP 서버

- 링크: https://smithery.ai/

사용한 도구 목록은 아래와 같습니다.

- Sequential Thinking: https://smithery.ai/server/@smithery-ai/server-sequential-thinking
  - 구조화된 사고 프로세스를 통해 역동적이고 성찰적인 문제 해결을 위한 도구를 제공하는 MCP 서버
- Desktop Commander: https://smithery.ai/server/@wonderwhy-er/desktop-commander
  - 다양한 편집 기능으로 터미널 명령을 실행하고 파일을 관리하세요. 코딩, 셸 및 터미널, 작업 자동화

**참고**

- smithery 에서 제공하는 도구를 JSON 형식으로 가져올때, 아래의 예시처럼 `"transport": "stdio"` 로 꼭 설정해야 합니다.

In [22]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [4]:
!pip install smithery mcp

In [5]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# LLM 모델 초기화
model = ChatOpenAI(
    api_key="ollama",
    model="llama3.2:1b",
    base_url="http://localhost:11434/v1",
    temperature=0,
)

# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "server-sequential-thinking": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@smithery-ai/server-sequential-thinking",
                "--key",
                "a59ae038-1dcc-48ec-a2bb-479ed58f8e0e",
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        "desktop-commander": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@wonderwhy-er/desktop-commander",
                "--key",
                "a59ae038-1dcc-48ec-a2bb-479ed58f8e0e",
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        "document-retriever": {
            "command": "/Users/hyunoozzing/.pyenv/versions/3.11.11/envs/llm/bin/python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
    }
)


# 2. 명시적으로 연결 초기화
await client.__aenter__()

CancelledError: 

https://smithery.ai에 있는 mcp 서버들은 현재 실행하려면 Claude, Cursor와 같은 llm모델을 사용해야만 호출해서 사용할 수 있는 것으로 보임.

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [19]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig


agent = create_react_agent(model, client.get_tools(), checkpointer=MemorySaver())

`Desktop Commander` 도구를 사용하여 터미널 명령을 실행합니다.

In [20]:
await astream_graph(
    agent,
    {
        "messages": "현재 경로를 포함한 하위 폴더 구조를 tree 로 그려줘. 단, .venv 폴더는 제외하고 출력해줘."
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
현재 경로와 하위 폴더 구조를 tree 형태로 그려드리겠습니다. `.venv` 폴더는 제외하고 출력하겠습니다.

이를 위해 `execute_command` 함수를 사용하여 tree 명령어를 실행하겠습니다. `-I` 옵션을 사용하면 특정 패턴을 제외할 수 있습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Command started with PID 5105
Initial output:
.
|____.git
| |____hooks
| |____info
| |____logs
| | |____refs
| | | |____heads
| | | |____remotes
| | | | |____origin
| |____objects
| | |____03
| | |____18
| | |____2d
| | |____36
| | |____3c
| | |____42
| | |____48
| | |____4d
| | |____59
| | |____5d
| | |____6f
| | |____75
| | |____93
| | |____96
| | |____99
| | |____9f
| | |____a7
| | |____a9
| | |____aa
| | |____b5
| | |____c0
| | |____c1
| | |____c8
| | |____cb
| | |____e2
| | |____e3
| | |____e4
| | |____e7
| | |____ea
| | |____f0
| | |____f7
| | |____info
| | |____pack
| |____refs
| | |____heads
| | |____remotes
| | | |____origin
| | |____tags
|____assets
|____data

🔄 Node: agent 🔄
- - - - - - - - - - - 

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-7325ff0e-cf97-471f-96ef-0ab2fc0ba215', usage_metadata={'input_tokens': 0, 'output_tokens': 321, 'total_tokens': 321, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 9,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:94e82fea-342b-2f07-08b0-432160e4029c',
  'checkpoint_ns': 'agent:94e82fea-342b-2f07-08b0-432160e4029c',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}

이번에는 `Sequential Thinking` 도구를 사용하여 비교적 복잡한 작업을 수행할 수 있는지 확인합니다.

In [21]:
await astream_graph(
    agent,
    {
        "messages": (
            "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 관련 내용을 검색하고 "
            "`Sequential Thinking` 도구를 사용해서 보고서를 작성해줘."
        )
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI 관련 내용을 검색하고 보고서를 작성해드리겠습니다. 먼저 `retrieve` 도구를 사용하여 관련 정보를 검색한 후, `sequentialthinking` 도구를 활용해 체계적인 보고서를 작성하겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약,

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-467bb435-0acf-42a3-8d47-601b8775393a', usage_metadata={'input_tokens': 0, 'output_tokens': 2247, 'total_tokens': 2247, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 32,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:8e9e85a9-cd93-4316-29f0-663a875bec41',
  'checkpoint_ns': 'agent:8e9e85a9-cd93-4316-29f0-663a875bec41',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}